In [2]:
import sys
import os
import fastText

from validation import compute_f1
import conlleval

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

import shutil, json

In [ ]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=2

In [3]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

In [4]:
testSentences = utils.get_sentences_germeval('../../Resources/GermEVAL/NER-de-test.tsv')
print(len(testSentences))


5100


In [5]:
models.ft = fastText.load_model("../../fastText/wiki.de.bin")

In [6]:
def load_model_indexes(indexes_file):
    indexMappings = json.load(open(indexes_file, "r"))
    models.idx2Label = {int(k):v for k,v in indexMappings[0].items()}
    models.label2Idx = indexMappings[1]
    models.char2Idx = indexMappings[2]
    models.case2Idx = indexMappings[3]
    models.max_sequence_length = 56

# Test Model

In [7]:
load_model_indexes('model_lstm_germeval_v2.0.h5.indexes')
finalmodel = load_model('model_lstm_germeval_v2.0.h5', custom_objects=create_custom_objects())

In [8]:
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

(0.8325358851674641, 0.8167691809647135, 0.8245771713375276)


# Official script eval CoNLL

In [ ]:
import conlleval

eval_file = 'test_pl.tsv'
write_conll_file(true_labels, pred_labels, testSentences, models.idx2Label, eval_file) 
p,r,f = evaluate_conll_file(eval_file) 

In [ ]:
print(p)
print(r)
print(f)

## Averages

In [ ]:
import numpy as np

In [ ]:
file_base_name = 'model_lstm_germeval_v2.'
eval_file = 'test_pl.tsv'
all_f1_scores = []

for run_i in range(10):
    model_file = file_base_name + str(run_i) + '.h5'
    index_file = file_base_name + str(run_i) + '.h5.indexes'
    if not os.path.isfile(model_file):
        print(model_file + ' not found')
        continue
    if not os.path.isfile(index_file):
        print(model_file + ' not found')
        continue
    
    print('Evaluating model ' + model_file)
    
    load_model_indexes(index_file)
    finalmodel = load_model(model_file, custom_objects=create_custom_objects())
    
    true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
    
    conlleval.write_conll_file(true_labels, pred_labels, testSentences, models.idx2Label, eval_file) 
    p,r,f = conlleval.evaluate_conll_file(eval_file) 
    all_f1_scores.append(f)

print(all_f1_scores)
print(np.mean(all_f1_scores))

In [ ]:
# import importlib
# importlib.reload(conlleval)
# importlib.reload(utils)

# Official script eval Germeval

In [9]:
idx_1st = models.idx2Label

In [10]:
print(models.label2Idx)
print(idx_1st)

{'B-ORGderiv': 2, 'B-PER': 10, 'PADDING_TOKEN': 0, 'I-ORGpart': 5, 'I-LOCpart': 17, 'B-OTHpart': 24, 'I-OTH': 4, 'B-ORG': 18, 'I-LOCderiv': 22, 'B-PERderiv': 11, 'B-LOC': 12, 'I-OTHderiv': 20, 'I-ORG': 15, 'B-OTH': 25, 'I-PER': 19, 'B-PERpart': 23, 'I-ORGderiv': 3, 'I-PERpart': 13, 'I-OTHpart': 7, 'O': 9, 'B-ORGpart': 14, 'I-PERderiv': 8, 'B-LOCderiv': 21, 'I-LOC': 1, 'B-LOCpart': 16, 'B-OTHderiv': 6}
{0: 'PADDING_TOKEN', 1: 'I-LOC', 2: 'B-ORGderiv', 3: 'I-ORGderiv', 4: 'I-OTH', 5: 'I-ORGpart', 6: 'B-OTHderiv', 7: 'I-OTHpart', 8: 'I-PERderiv', 9: 'O', 10: 'B-PER', 11: 'B-PERderiv', 12: 'B-LOC', 13: 'I-PERpart', 14: 'B-ORGpart', 15: 'I-ORG', 16: 'B-LOCpart', 17: 'I-LOCpart', 18: 'B-ORG', 19: 'I-PER', 20: 'I-OTHderiv', 21: 'B-LOCderiv', 22: 'I-LOCderiv', 23: 'B-PERpart', 24: 'B-OTHpart', 25: 'B-OTH'}


In [11]:
testSentences_2nd = utils.get_sentences_germeval('../../Resources/GermEVAL/NER-de-test.tsv', level2=True)
load_model_indexes('model_lstm_germeval_2nd-level.h5.indexes')
finalmodel_2nd = load_model('model_lstm_germeval_2nd-level.h5', custom_objects=create_custom_objects())
true_labels_2nd, pred_labels_2nd = utils.predict_sequences(finalmodel_2nd, testSentences_2nd)
print(compute_f1(pred_labels_2nd, true_labels_2nd, models.idx2Label))
idx_2nd = models.idx2Label

(0.5807127882599581, 0.537864077669903, 0.5584677419354839)


In [12]:
eval_file = 'germeval_outer_inner.tsv'
conlleval.write_germeval_file(true_labels, pred_labels, true_labels_2nd, pred_labels_2nd, testSentences, idx_1st, idx_2nd, eval_file) 